<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session8_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 8 (Syntactic Parser) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [5]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/


Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 393 (delta 139), reused 89 (delta 53), pack-reused 190
Receiving objects: 100% (393/393), 327.88 KiB | 4.82 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [7]:
%load_ext autoreload
%autoreload 2

In [6]:
import nltk
from nltk import CFG, ChartParser
!pip install svgling
import svgling

In [ ]:
from scripts.compute_metrics import ComputeMetrics
from scripts.text_preprocessing import TextPreprocessing
from complementary_material.textserver import TextServer
from scripts.utils import pretty_table
from scipy.stats import pearsonr
import pandas as pd
pd.set_option("display.precision", 4)

## TODO

- Consider the following sentence:
Lazy cats play with mice.

- Expand the grammar of the example related to non-probabilistic chart parsers in order to subsume this new sentence.

- Perform the constituency parsing using a BottomUpChartParser, a BottomUpLeftCornerChartParser and a LeftCornerChartParser.

- For each one of them, provide the resulting tree, the number of edges and the list of explored edges.

- Which parser is the most efficient for parsing the sentence?

- Which edges are filtered out by each parser and why?

In [8]:
sentence="Lazy cats play with mice."

In [2]:
grammar = CFG.fromstring('''
  NP  -> NNS | JJ NNS | NP CC NP
  NNS -> "cats" | "dogs" | "mice" | NNS CC NNS
  JJ  -> "big" | "small"
  CC  -> "and" | "or"
  ''')
parser = ChartParser(grammar, trace=1)
parse = parser.chart_parse(['small', 'cats', 'and', 'mice'])


|.  small  .   cats  .   and   .   mice  .|
|[---------]         .         .         .| [0:1] 'small'
|.         [---------]         .         .| [1:2] 'cats'
|.         .         [---------]         .| [2:3] 'and'
|.         .         .         [---------]| [3:4] 'mice'
|[---------]         .         .         .| [0:1] JJ -> 'small' *
|[--------->         .         .         .| [0:1] NP -> JJ * NNS
|.         [---------]         .         .| [1:2] NNS -> 'cats' *
|.         [---------]         .         .| [1:2] NP -> NNS *
|.         [--------->         .         .| [1:2] NNS -> NNS * CC NNS
|[-------------------]         .         .| [0:2] NP -> JJ NNS *
|[------------------->         .         .| [0:2] NP -> NP * CC NP
|.         [--------->         .         .| [1:2] NP -> NP * CC NP
|.         .         [---------]         .| [2:3] CC -> 'and' *
|.         [------------------->         .| [1:3] NNS -> NNS CC * NNS
|[----------------------------->         .| [0:3] NP -> NP CC * NP

In [ ]:
parse.num_edges()

In [ ]:
parse.edges()

In [ ]:
ts = list(parse)

In [ ]:
print(ts[0])

In [ ]:
ts[0]

In [ ]:
ts[1]

In [ ]:
from nltk import TopDownChartParser
parser = nltk.TopDownChartParser(grammar)
parse = parser.parse(sent)